# 2. Classification

In [1]:
# QUESTO ANDRA' AGGIUNTO AGLI IMPORTI

# Run this code to make Jupyter print every
# printable statement and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# To visualize the data
import matplotlib.pyplot as plt

# Generic libraries
import seaborn as sns
import pandas as pd
import numpy as np

# Regression models
import sklearn
import scipy
from scipy.stats import *
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV #split the data into training and test
from sklearn.linear_model import LinearRegression #linear regression
from sklearn.preprocessing import PolynomialFeatures #for polynomial regression
from sklearn.metrics import r2_score, mean_squared_error

# 5-folds crossvalidation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, classification_report, roc_auc_score
from sklearn.pipeline import Pipeline

#classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

## Preprocessing

In [2]:
# QUESTO ANDRA' CANCELLATO

training_set = pd.read_csv("training_set_complete.csv")
test_set = pd.read_csv("test_set_complete.csv")
#training_set = training_set.drop(columns=['Unnamed: 0'])
#test_set = test_set.drop(columns=['Unnamed: 0'])
training_set['Revenue'] = training_set['Revenue'].astype(int)
test_set['Revenue'] = test_set['Revenue'].astype(int)
#training_set['Revenue'].value_counts()
#test_set['Revenue'].value_counts()
categorical_features = ['Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June',
       'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep',
       'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3',
       'OperatingSystems_4', 'OperatingSystems_5', 'OperatingSystems_6',
       'OperatingSystems_7', 'OperatingSystems_8', 'Browser_1', 'Browser_2',
       'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7',
       'Browser_8', 'Browser_9', 'Browser_10', 'Browser_11', 'Browser_12',
       'Browser_13', 'Region_1', 'Region_2', 'Region_3', 'Region_4',
       'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9',
       'TrafficType_1', 'TrafficType_2', 'TrafficType_3', 'TrafficType_4',
       'TrafficType_5', 'TrafficType_6', 'TrafficType_7', 'TrafficType_8',
       'TrafficType_9', 'TrafficType_10', 'TrafficType_11', 'TrafficType_12',
       'TrafficType_13', 'TrafficType_14', 'TrafficType_15', 'TrafficType_16',
       'TrafficType_17', 'TrafficType_18', 'TrafficType_19', 'TrafficType_20',
       'VisitorType_New_Visitor', 'VisitorType_Other',
       'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']
#training_set[categorical_features] = training_set[categorical_features].astype('category')
#test_set[categorical_features] = test_set[categorical_features].astype('category')

In [3]:
# QUESTO E' QUELLO CON GLI EXIT RATES 
# FARE COPIA DEL DATASET CHE OTTENIAMO ALLA FINE DI REGRESSION CON .COPY
# TRAINING E TEST

X_train_full = training_set.drop(columns=['Revenue'])
X_test_full = test_set.drop(columns=['Revenue'])
y_train = training_set['Revenue']
y_test = test_set['Revenue']

### Variance selection

SPIEGAZIONE 9

In [14]:
# FACCIAMO QUESTO STEP PER EVITARE PROBLEMI DI CONVERGENZA NEL MODELLO
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature data X
numeric_features = ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_features = ['Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May', 
                        'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Browser_1', 'Browser_2', 'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7', 'Browser_8', 'Browser_10', 'Browser_13', 'Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5',
                        'Region_6', 'Region_7', 'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_2', 
                        'TrafficType_3', 'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_7', 
                        'TrafficType_8', 'TrafficType_9', 'TrafficType_10', 'TrafficType_11', 'TrafficType_13', 
                        'TrafficType_15', 'TrafficType_19', 'TrafficType_20', 'VisitorType_New_Visitor', 'VisitorType_Other', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train_full[numeric_features]))
test_scaled = pd.DataFrame(scaler.fit_transform(X_test_full[numeric_features]))
train_scaled.columns = X_train_full[numeric_features].columns
test_scaled.columns = X_test_full[numeric_features].columns
X_train_scaled = pd.concat([train_scaled,X_train_full[categorical_features]],axis=1)
X_test_scaled = pd.concat([test_scaled,X_test_full[categorical_features]],axis=1)

In [15]:
from sklearn.feature_selection import VarianceThreshold

threshold = 0.0075  # Soglia di varianza desiderata
variance_selector = VarianceThreshold(threshold=threshold)

# Applicazione della selezione della varianza sul dataset

X_train_scaled = X_train_scaled.loc[:, variance_selector.fit(X_train_scaled).get_support()]
columns_remaining = X_train_scaled.columns.tolist()

X_test_scaled = X_test_scaled[columns_remaining]
print(X_train_scaled.columns.tolist())
len(columns_remaining)

['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4', 'Browser_1', 'Browser_2', 'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_8', 'Browser_10', 'Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_2', 'TrafficType_3', 'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_8', 'TrafficType_10', 'TrafficType_11', 'TrafficType_13', 'TrafficType_20', 'VisitorType_New_Visitor', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']


56

### Functions and algorithms

### Feature selection: Forward stepwise selection

SPIEGAZIONE 10

In [43]:
# VA SOSTITUITO NELLA REGRESSION

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def get_evaluator(scorer):
    def evaluator(model, X, y, trained=False):
        if not trained:
            model = model.fit(X, y)
        score = scorer(model, X, y)
        return model, score
    return evaluator   

def get_cv_evaluator(scorer, cv=3):
    def evaluator(model, X, y, trained=False):            
        scores = cross_val_score(model, X, y, scoring=scorer, cv=cv)
        if not trained:
            model = model.fit(X, y)
        return model, np.mean(scores)
    return evaluator

# Definition of helper functions to generalize forward stepwise selection for different algorithms
def get_logistic_model(solver, penalty, max_iter=1000):
    def logistic_model():
        model = LogisticRegression(solver=solver, penalty=penalty,max_iter=max_iter)
        return model
    return logistic_model

def get_LDA_model():
    def LDA_model():
        model = LDA()
        return model
    return LDA_model

def get_QDA_model():
    def QDA_model():
        model = QDA()
        return model
    return QDA_model

def get_linear_regression_model(fit_intercept=True):
    def linear_regression_model():
        model = LinearRegression(fit_intercept=fit_intercept)
        return model
    return linear_regression_model

def get_SVM_model(kernel,C,max_iter=1000):
    def SVM_model():
        if kernel == 'linear':
            model = LinearSVC(C=C,max_iter=max_iter)
        else:
            model = SVC(kernel=kernel,C=C,max_iter=max_iter)
        return model
    return SVM_model


def forward_selection(Xtrain_pd, ytrain, model_chosen,
                      candidates_evaluator, candidates_argbest, # Metric to be used at 2.b
                      subsets_evaluator, subsets_argbest,       # Metric to be used at 3
                      candidates_scorer_name=None,  # Name of 2. figure
                      subsets_scorer_name=None,     # Name of 3. figure
                      verbose=True, weight_step3=0):   
    
    
    # Global variable init
    # ====================
    num_features = Xtrain_pd.shape[-1]
    best_candidate_metric = []
    # subsets_* are lists containing one value for each Mk model (the best of the Mk candidates)
    subsets_metric = []        # The best metric of each subset of dimension 'dim'
    subsets_best_features = [] # The best features combination in each subset of dimension 'dim'
    # A figure to keep track of candidates scores in each Mk subset
    num_evaluations = 0        # A conter to keep track of the total number of trials
    
    selected_features = []
    all_features = Xtrain_pd.columns
    
    
    # 1. Train M0
    # ===========
    model = DummyRegressor()
    # Compute (2.b) metrics
    model, score = candidates_evaluator(model, Xtrain_pd[[]], ytrain)
    best_candidate_metric.append(score)
    subsets_best_features.append([])
    # Compute metric for step 3.
    _, score = subsets_evaluator(model, Xtrain_pd[[]], ytrain, trained=True)
    subsets_metric.append(score)
    
    
    for dim in range(num_features):
        candidate_metrics = [] # metrics for all the models with dim features
        candidate_models = []  # models with dim features
        
        remaining_features = all_features.difference(selected_features)
        
        # fit all the models with k features
        for new_column in remaining_features:
            Xtrain_sub = Xtrain_pd[selected_features+[new_column]].to_numpy()
            #model = possible_models[model_chosen]
            model = model_chosen()
            model, score = candidates_evaluator(model, Xtrain_sub, ytrain)
            candidate_models.append(model)
            candidate_metrics.append(score)
            num_evaluations += 1
            
        
        idx_best_candidate = candidates_argbest(candidate_metrics) # select the best Mk model
        selected_features.append(remaining_features[idx_best_candidate]) # Update selected feature
        best_candidate_metric.append(candidate_metrics[idx_best_candidate]) # Save best candidate features
        best_features = selected_features.copy()
        subsets_best_features.append(best_features)
        
        
        # Compute metric for the final step -> comparison of all the best models
        best_subset_model = candidate_models[idx_best_candidate] # save the best model
        best_subset_Xtrain = Xtrain_pd[best_features].to_numpy()
        _, score = subsets_evaluator(best_subset_model, best_subset_Xtrain, ytrain, trained=True)
        subsets_metric.append(score) #computing the metrics for the training set
        num_evaluations += weight_step3 
        
        if verbose:
            print("............")
            print("Best model (M{}) with {} features: {}".format(dim+1, dim+1, best_features))
            print("M{} subset score (3.): {}".format(dim+1, score))
        
    # choose the best candidates
    best_subset_idx = subsets_argbest(subsets_metric)
    best_features = subsets_best_features[best_subset_idx]
    
    if verbose:
        print("\n Best configuration has {} features".format(best_subset_idx))
        print("Features: {}".format(subsets_best_features[best_subset_idx]))
        print("Total number of trained models:", num_evaluations)
    
    # Complete the subsets_fig figure by plotting
    # a line connecting all best candidate score
    best_candidate_score_idx = candidates_argbest(best_candidate_metric)
    return best_features
    

#### Accuracy

In [5]:
def accuracy(y_pred, y_true):
    return (y_pred == y_true).mean()

def calculate_sensitivity_specificity(confusion_matrix):
    # Extract values from the confusion matrix
    TN, FP, FN, TP = confusion_matrix.ravel()

    # Calculate Sensitivity (Recall)
    sensitivity = TP / (TP + FN)

    # Calculate Specificity
    specificity = TN / (TN + FP)

    return sensitivity, specificity

#### Evaluation metrics

In [7]:
from sklearn.metrics import confusion_matrix

def classification_metrics_training(model, model_name, X_train, y_train, cv):
    
    # Sensitivity
    sensitivity_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='recall', n_jobs=-1)))
    
    # Specificity
    specificity_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='precision', n_jobs=-1)))
    
    # Accuracy
    accuracy_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)))
                         
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy_train],
        'Sensitivity': [sensitivity_train],
        'Specificity': [specificity_train],
    }
    return pd.DataFrame(data)                    

In [8]:
def classification_metrics_test(model, model_name,X_train,y_train,X_test,y_test,threshold):
    
    model = model.fit(X_train,y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    ypred = (y_pred_proba > threshold).astype(int)
    
    cm = confusion_matrix(y_test.to_numpy(), np.array(ypred))
    sensitivity_test, specificity_test = calculate_sensitivity_specificity(cm)
    
    accuracy_test = accuracy(ypred, y_test)
    
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy_test],
        'Sensitivity': [sensitivity_test],
        'Specificity': [specificity_test],
    }
    return pd.DataFrame(data), cm

In [10]:
# Compute the empirical threshold
empirical_threshold = y_train.mean()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def classification_metrics_empirical_threshold(model, model_name, X_train, y_train,threshold=0.5):
    
    model = model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_train)[:, 1]
    y_pred = (y_pred_proba > threshold).astype(int)

    accuracy = accuracy_score(y_train, y_pred)
    specificity = precision_score(y_train, y_pred)
    sensitivity = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy],
        'Sensitivity': [sensitivity],
        'Specificity': [specificity],
    }
    return pd.DataFrame(data) 

## Logistic regression

- Features selection: forward stepwise selection. Models are evaluated with cross-validated accuracy.
- Fit the best model with the selected features.
- Evaluate the performances computing accuracy, sensitivity and specificity.
- Try to improve sensitivity setting the empirical threshold, balancing the trade-off.

In [16]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_logistic = forward_selection(X_train_scaled, y_train, get_logistic_model('newton-cg', 'none', max_iter=1000),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_logistic

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8802981889406984
............
Best model (M2) with 2 features: ['PageValues', 'VisitorType_New_Visitor']
M2 subset score (3.): 0.8823527107421105
............
Best model (M3) with 3 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3']
M3 subset score (3.): 0.8830015347960183
............
Best model (M4) with 4 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10']
M4 subset score (3.): 0.8837583500211948
............
Best model (M5) with 5 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8']
M5 subset score (3.): 0.8838665165977228
............
Best model (M6) with 6 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1']
M6 subset score (3.): 0.8830015347960183
............
Best model (M7) with 7 features: ['PageValues', 'VisitorType_New_Visitor', 'Traff

............
Best model (M28) with 28 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1', 'Browser_2', 'TrafficType_11', 'TrafficType_6', 'OperatingSystems_4', 'Browser_10', 'Region_6', 'Region_5', 'VisitorType_Returning_Visitor', 'Browser_3', 'Browser_5', 'Region_7', 'Browser_8', 'Month_June', 'TrafficType_20', 'TrafficType_1', 'Month_Feb', 'BounceRates', 'TrafficType_4', 'TrafficType_8', 'Month_Oct', 'Browser_4', 'Region_2']
M28 subset score (3.): 0.8841908409220471
............
Best model (M29) with 29 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1', 'Browser_2', 'TrafficType_11', 'TrafficType_6', 'OperatingSystems_4', 'Browser_10', 'Region_6', 'Region_5', 'VisitorType_Returning_Visitor', 'Browser_3', 'Browser_5', 'Region_7', 'Browser_8', 'Month_June', 'TrafficType_20', 'TrafficType_1', 'Month_Feb', 'BounceRates', 'TrafficType_4', 'TrafficType_8', 'Month_Oct', '

............
Best model (M42) with 42 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1', 'Browser_2', 'TrafficType_11', 'TrafficType_6', 'OperatingSystems_4', 'Browser_10', 'Region_6', 'Region_5', 'VisitorType_Returning_Visitor', 'Browser_3', 'Browser_5', 'Region_7', 'Browser_8', 'Month_June', 'TrafficType_20', 'TrafficType_1', 'Month_Feb', 'BounceRates', 'TrafficType_4', 'TrafficType_8', 'Month_Oct', 'Browser_4', 'Region_2', 'Region_4', 'Month_Sep', 'Region_3', 'TrafficType_5', 'Browser_6', 'Month_Aug', 'OperatingSystems_1', 'OperatingSystems_3', 'ExitRates', 'Month_Jul', 'OperatingSystems_2', 'Month_Dec', 'Weekend_False', 'SpecialDay']
M42 subset score (3.): 0.8830011255170801
............
Best model (M43) with 43 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1', 'Browser_2', 'TrafficType_11', 'TrafficType_6', 'OperatingSystems_4', 'Browser_10', 'Region_6', 'Regi

............
Best model (M52) with 52 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Region_8', 'Browser_1', 'Browser_2', 'TrafficType_11', 'TrafficType_6', 'OperatingSystems_4', 'Browser_10', 'Region_6', 'Region_5', 'VisitorType_Returning_Visitor', 'Browser_3', 'Browser_5', 'Region_7', 'Browser_8', 'Month_June', 'TrafficType_20', 'TrafficType_1', 'Month_Feb', 'BounceRates', 'TrafficType_4', 'TrafficType_8', 'Month_Oct', 'Browser_4', 'Region_2', 'Region_4', 'Month_Sep', 'Region_3', 'TrafficType_5', 'Browser_6', 'Month_Aug', 'OperatingSystems_1', 'OperatingSystems_3', 'ExitRates', 'Month_Jul', 'OperatingSystems_2', 'Month_Dec', 'Weekend_False', 'SpecialDay', 'Weekend_True', 'Informational', 'Informational_Duration', 'Month_Nov', 'Month_Mar', 'Month_May', 'Administrative_Duration', 'Region_1', 'Region_9', 'TrafficType_2']
M52 subset score (3.): 0.8825686930846477
............
Best model (M53) with 53 features: ['PageValues', 'VisitorType_New_Visit

['PageValues',
 'VisitorType_New_Visitor',
 'TrafficType_3',
 'TrafficType_10',
 'Region_8',
 'Browser_1',
 'Browser_2',
 'TrafficType_11',
 'TrafficType_6',
 'OperatingSystems_4',
 'Browser_10',
 'Region_6',
 'Region_5',
 'VisitorType_Returning_Visitor',
 'Browser_3',
 'Browser_5',
 'Region_7',
 'Browser_8',
 'Month_June',
 'TrafficType_20',
 'TrafficType_1',
 'Month_Feb',
 'BounceRates',
 'TrafficType_4',
 'TrafficType_8',
 'Month_Oct',
 'Browser_4',
 'Region_2']

In [17]:
model_logistic = LogisticRegression(solver="newton-cg",penalty="none",max_iter=1000)
train_scores_logistic = classification_metrics_training(model_logistic, "LogisticRegression", X_train_scaled[features_logistic], y_train,cv)
train_scores_logistic

,Model,Accuracy,Sensitivity,Specificity
0,LogisticRegression,0.884191,0.360713,0.776204


In [18]:
train_scores_logistic_empirical = classification_metrics_empirical_threshold(model_logistic, 'LogisticRegression', X_train_scaled[features_logistic], y_train,empirical_threshold)
train_scores_logistic_empirical

,Model,Accuracy,Sensitivity,Specificity
0,LogisticRegression,0.861159,0.697432,0.542364


## LDA

- Features selection: forward stepwise selection. Models are evaluated with cross-validated accuracy.
- Fit the best model with the selected features.
- Evaluate the performances computing accuracy, sensitivity and specificity.
- Try to improve sensitivity setting the empirical threshold, balancing the trade-off.

In [19]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_LDA = forward_selection(X_train_scaled, y_train, get_LDA_model(),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_LDA

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8766217531755661
............
Best model (M2) with 2 features: ['PageValues', 'ExitRates']
M2 subset score (3.): 0.8783517167789748
............
Best model (M3) with 3 features: ['PageValues', 'ExitRates', 'TrafficType_3']
M3 subset score (3.): 0.8782437256077061
............
Best model (M4) with 4 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2']
M4 subset score (3.): 0.8785680499320303
............
Best model (M5) with 5 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2']
M5 subset score (3.): 0.8792168739859385
............
Best model (M6) with 6 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates']
M6 subset score (3.): 0.8794332071389942
............
Best model (M7) with 7 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRat

............
Best model (M29) with 29 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'Browser_10', 'Browser_3', 'Month_Feb', 'Month_Sep', 'Region_7', 'Browser_8', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Browser_4', 'TrafficType_6', 'OperatingSystems_4', 'Region_8', 'Month_Aug', 'Month_June', 'Region_5', 'Region_6', 'Region_4', 'Month_Oct', 'Browser_2', 'Browser_1']
M29 subset score (3.): 0.8800822065981613
............
Best model (M30) with 30 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'Browser_10', 'Browser_3', 'Month_Feb', 'Month_Sep', 'Region_7', 'Browser_8', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Browser_4', 'TrafficType_6', 'OperatingSystems_4', 'Region_8', 'Month_Aug', 'Month_June', 'Region_5', 'Region_6', 'Region_4', 'Month_Oct', 'Browser_2', 'Browser_1', 'OperatingSystems_1'

............
Best model (M43) with 43 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'Browser_10', 'Browser_3', 'Month_Feb', 'Month_Sep', 'Region_7', 'Browser_8', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Browser_4', 'TrafficType_6', 'OperatingSystems_4', 'Region_8', 'Month_Aug', 'Month_June', 'Region_5', 'Region_6', 'Region_4', 'Month_Oct', 'Browser_2', 'Browser_1', 'OperatingSystems_1', 'OperatingSystems_3', 'Region_1', 'TrafficType_4', 'Browser_6', 'VisitorType_New_Visitor', 'TrafficType_10', 'Month_Dec', 'VisitorType_Returning_Visitor', 'TrafficType_20', 'Administrative_Duration', 'Weekend_False', 'Weekend_True', 'Month_Jul']
M43 subset score (3.): 0.8805144636253344
............
Best model (M44) with 44 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'Browser_10', 'Browser_3', 'Month_Feb', 'Month_Se

............
Best model (M53) with 53 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'Browser_10', 'Browser_3', 'Month_Feb', 'Month_Sep', 'Region_7', 'Browser_8', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Browser_4', 'TrafficType_6', 'OperatingSystems_4', 'Region_8', 'Month_Aug', 'Month_June', 'Region_5', 'Region_6', 'Region_4', 'Month_Oct', 'Browser_2', 'Browser_1', 'OperatingSystems_1', 'OperatingSystems_3', 'Region_1', 'TrafficType_4', 'Browser_6', 'VisitorType_New_Visitor', 'TrafficType_10', 'Month_Dec', 'VisitorType_Returning_Visitor', 'TrafficType_20', 'Administrative_Duration', 'Weekend_False', 'Weekend_True', 'Month_Jul', 'SpecialDay', 'Informational', 'Region_2', 'Region_9', 'Informational_Duration', 'TrafficType_1', 'TrafficType_8', 'Month_Mar', 'ProductRelated_Duration', 'Administrative']
M53 subset score (3.): 0.8771627029950448
............
Best model (M54) with 54 features: 

['PageValues',
 'ExitRates',
 'TrafficType_3',
 'OperatingSystems_2',
 'TrafficType_2',
 'BounceRates',
 'Region_3',
 'Browser_5',
 'Browser_10',
 'Browser_3',
 'Month_Feb',
 'Month_Sep',
 'Region_7',
 'Browser_8',
 'TrafficType_13',
 'TrafficType_11',
 'TrafficType_5',
 'Browser_4',
 'TrafficType_6',
 'OperatingSystems_4',
 'Region_8',
 'Month_Aug',
 'Month_June',
 'Region_5',
 'Region_6',
 'Region_4',
 'Month_Oct',
 'Browser_2',
 'Browser_1',
 'OperatingSystems_1',
 'OperatingSystems_3',
 'Region_1',
 'TrafficType_4',
 'Browser_6',
 'VisitorType_New_Visitor',
 'TrafficType_10',
 'Month_Dec']

In [20]:
model_LDA =LDA(store_covariance=True)
train_scores_LDA = classification_metrics_training(model_LDA, "LDA", X_train_scaled[features_LDA], y_train, cv)
train_scores_LDA

,Model,Accuracy,Sensitivity,Specificity
0,LDA,0.880839,0.32617,0.782223


In [21]:
train_scores_LDA_empirical = classification_metrics_empirical_threshold(model_LDA, 'LDA', X_train_scaled[features_LDA], y_train,empirical_threshold)
train_scores_LDA_empirical

,Model,Accuracy,Sensitivity,Specificity
0,LDA,0.891003,0.54823,0.688753


## QDA

- Features selection: forward stepwise selection. Models are evaluated with cross-validated accuracy.
- Fit the best model with the selected features.
- Evaluate the performances computing accuracy, sensitivity and specificity.
- Try to improve sensitivity setting the empirical threshold, balancing the trade-off.

In [22]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_QDA = forward_selection(X_train_scaled, y_train, get_QDA_model(),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=False, weight_step3=cv.n_splits)
features_QDA

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

...

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Valentina\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\V

['PageValues',
 'BounceRates',
 'VisitorType_New_Visitor',
 'OperatingSystems_3',
 'OperatingSystems_1']

In [23]:
model_QDA = QDA(store_covariance=True)
train_scores_QDA = classification_metrics_training(model_QDA, "QDA", X_train_scaled[features_QDA], y_train, cv)
train_scores_QDA

,Model,Accuracy,Sensitivity,Specificity
0,QDA,0.890462,0.46891,0.731185


In [24]:
train_scores_QDA_empirical = classification_metrics_empirical_threshold(model_QDA, 'QDA', X_train_scaled[features_QDA], y_train, empirical_threshold)
train_scores_QDA_empirical

,Model,Accuracy,Sensitivity,Specificity
0,QDA,0.815852,0.691881,0.441933


## KNN

SPIEGAZIONE 11

### KNN with all features

In [25]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [27]:
model_KNN_all = KNeighborsClassifier()
params = {'n_neighbors': range(1, 15)}
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cv_KNN_all = GridSearchCV(model_KNN_all, params, refit=True, cv=cv, 
                  scoring=make_scorer(accuracy))
cv_KNN_all.fit(X_train_scaled, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 15)},
             scoring=make_scorer(accuracy))

In [28]:
model_KNN_all = KNeighborsClassifier(n_neighbors=cv_KNN_all.best_params_['n_neighbors'])
train_scores_KNN_all = classification_metrics_training(model_KNN_all, "KNN", X_train_scaled, y_train, cv)
train_scores_KNN_all

,Model,Accuracy,Sensitivity,Specificity
0,KNN,0.875433,0.29694,0.754981


In [29]:
train_scores_KNN_all_empirical = classification_metrics_empirical_threshold(model_KNN_all, 'KNN', X_train_scaled, y_train, empirical_threshold)
train_scores_KNN_all_empirical

,Model,Accuracy,Sensitivity,Specificity
0,KNN,0.827747,0.795975,0.468929


### With only numeric features

In [32]:
model_KNN_numeric = KNeighborsClassifier()
params = {'n_neighbors': range(1, 50)}
cv_KNN_numeric = GridSearchCV(model_KNN_numeric, params, refit=True, cv=cv, 
                  scoring=make_scorer(accuracy))
cv_KNN_numeric.fit(X_train_scaled[numeric_features], y_train)
model_KNN_numeric = KNeighborsClassifier(n_neighbors=cv_KNN_numeric.best_params_['n_neighbors'])


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)},
             scoring=make_scorer(accuracy))

In [33]:
train_scores_KNN_numeric = classification_metrics_training(model_KNN_numeric, "KNN", X_train_scaled[numeric_features], y_train, cv)
train_scores_KNN_numeric

,Model,Accuracy,Sensitivity,Specificity
0,KNN,0.888948,0.431234,0.74917


In [34]:
train_scores_KNN_numeric_empirical = classification_metrics_empirical_threshold(model_KNN_numeric, 'KNN', X_train_scaled[numeric_features], y_train, empirical_threshold)
train_scores_KNN_numeric_empirical

,Model,Accuracy,Sensitivity,Specificity
0,KNN,0.827206,0.815406,0.468688


## SVM

- Choice of the best kernel.
- Choice of the best C value.
- Features selection: forward stepwise selection. Models are evaluated with cross-validated accuracy.
- Fit the best model with the selected features.
- Evaluate the performances computing accuracy, sensitivity and specificity.
- Try to improve sensitivity setting the empirical threshold, balancing the trade-off.

In [1]:
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
score = []
for kern in kernels:
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    if kern == 'linear':
        model_SVM = LinearSVC()
    else:
        model_SVM = SVC(kernel=kern)
    scores = cross_val_score(model_SVM, X_train_scaled, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    score.append(np.mean(scores))
    print("Accuracy score for {} kernel: ".format(kern), np.mean(scores))

print("Best kernel: ", best_kernel)

NameError: name 'KFold' is not defined

In [53]:
C_s= [0.1,1,10,100]

score_c = []
   
for c in C_s:
    if best_kernel=='linear':
        model_SVM = LinearSVC(C=c)
    else:
        model_SVM = SVC(kernel=best_kernel)
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    scores_score = cross_val_score(model_SVM, X_train_scaled, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    score_c.append(np.mean(scores_score))
best_score_c = score_c.index(max(score_c))
   

best_c = C_s[best_score_c]
print("Best C value: ", best_c)

Best C value:  0.1


In [54]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_SVM = forward_selection(X_train_scaled, y_train, get_SVM_model(best_kernel, best_c, 500),
                  get_evaluator(make_scorer(accuracy)), np.argmax, 
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, 
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_SVM

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8788924911931943
............
Best model (M2) with 2 features: ['PageValues', 'BounceRates']
M2 subset score (3.): 0.8791087658778303
............
Best model (M3) with 3 features: ['PageValues', 'BounceRates', 'TrafficType_1']
M3 subset score (3.): 0.8795412567786824
............
Best model (M4) with 4 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11']
M4 subset score (3.): 0.8796493648867905
............
Best model (M5) with 5 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug']
M5 subset score (3.): 0.8797574729948987
............
Best model (M6) with 6 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9']
M6 subset score (3.): 0.8796494233552103
............
Best model (M7) with 7 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20']

............
Best model (M28) with 28 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20', 'TrafficType_2', 'Browser_3', 'Browser_8', 'Month_June', 'Browser_10', 'Weekend_True', 'OperatingSystems_3', 'VisitorType_New_Visitor', 'Region_2', 'Month_Oct', 'Region_3', 'Weekend_False', 'Region_8', 'VisitorType_Returning_Visitor', 'Region_5', 'Region_6', 'TrafficType_4', 'Month_Feb', 'Browser_1', 'Region_4', 'OperatingSystems_4']
M28 subset score (3.): 0.8807306798415506
............
Best model (M29) with 29 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20', 'TrafficType_2', 'Browser_3', 'Browser_8', 'Month_June', 'Browser_10', 'Weekend_True', 'OperatingSystems_3', 'VisitorType_New_Visitor', 'Region_2', 'Month_Oct', 'Region_3', 'Weekend_False', 'Region_8', 'VisitorType_Returning_Visitor', 'Region_5', 'Region_6', 'TrafficType_4', 'Month_Feb', 'Browser_1', 'Region_

............
Best model (M42) with 42 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20', 'TrafficType_2', 'Browser_3', 'Browser_8', 'Month_June', 'Browser_10', 'Weekend_True', 'OperatingSystems_3', 'VisitorType_New_Visitor', 'Region_2', 'Month_Oct', 'Region_3', 'Weekend_False', 'Region_8', 'VisitorType_Returning_Visitor', 'Region_5', 'Region_6', 'TrafficType_4', 'Month_Feb', 'Browser_1', 'Region_4', 'OperatingSystems_4', 'Browser_2', 'Browser_4', 'TrafficType_13', 'TrafficType_5', 'TrafficType_6', 'Month_Sep', 'Region_1', 'Region_7', 'TrafficType_8', 'Browser_5', 'OperatingSystems_1', 'TrafficType_10', 'OperatingSystems_2', 'Browser_6']
M42 subset score (3.): 0.8796494233552103
............
Best model (M43) with 43 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20', 'TrafficType_2', 'Browser_3', 'Browser_8', 'Month_June', 'Browser_10', 'Weekend_True', 'Op

............
Best model (M52) with 52 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Month_Aug', 'Region_9', 'TrafficType_20', 'TrafficType_2', 'Browser_3', 'Browser_8', 'Month_June', 'Browser_10', 'Weekend_True', 'OperatingSystems_3', 'VisitorType_New_Visitor', 'Region_2', 'Month_Oct', 'Region_3', 'Weekend_False', 'Region_8', 'VisitorType_Returning_Visitor', 'Region_5', 'Region_6', 'TrafficType_4', 'Month_Feb', 'Browser_1', 'Region_4', 'OperatingSystems_4', 'Browser_2', 'Browser_4', 'TrafficType_13', 'TrafficType_5', 'TrafficType_6', 'Month_Sep', 'Region_1', 'Region_7', 'TrafficType_8', 'Browser_5', 'OperatingSystems_1', 'TrafficType_10', 'OperatingSystems_2', 'Browser_6', 'TrafficType_3', 'Month_Jul', 'ExitRates', 'Informational_Duration', 'Month_Mar', 'Informational', 'Administrative_Duration', 'SpecialDay', 'Administrative', 'ProductRelated']
M52 subset score (3.): 0.8793252744361452
............
Best model (M53) with 53 features: ['PageValues', 'Bounce

['PageValues',
 'BounceRates',
 'TrafficType_1',
 'TrafficType_11',
 'Month_Aug',
 'Region_9',
 'TrafficType_20',
 'TrafficType_2',
 'Browser_3',
 'Browser_8',
 'Month_June',
 'Browser_10',
 'Weekend_True',
 'OperatingSystems_3',
 'VisitorType_New_Visitor',
 'Region_2',
 'Month_Oct',
 'Region_3',
 'Weekend_False',
 'Region_8',
 'VisitorType_Returning_Visitor',
 'Region_5',
 'Region_6',
 'TrafficType_4',
 'Month_Feb',
 'Browser_1',
 'Region_4',
 'OperatingSystems_4',
 'Browser_2',
 'Browser_4']

In [ ]:
model_SVM = LinearSVC()
model_SVM = model_SVM.fit(X_train_scaled[features_SVM],y_train)

In [ ]:
train_scores_SVM = classification_metrics_training(model_SVM, "SVM", X_train_scaled[features_SVM], y_train, cv)
train_scores_SVM

## Algorithm comparison

### Metric comparison

#### Threshold = 0.5

In [ ]:
metrics_df = pd.concat([train_scores_logistic,train_scores_LDA,train_scores_QDA,train_scores_KNN_all,train_scores_KNN_numeric,train_scores_SVM],axis=0)
metrics_df

#### Empirical threshold

In [ ]:
metrics_df_empirical = pd.concat([train_scores_logistic_empirical,train_scores_LDA_empirical,train_scores_QDA_empirical,train_scores_KNN_all_empirical,train_scores_KNN_numeric_empirical],axis=0)
metrics_df_empirical

In [ ]:
from sklearn.metrics import roc_curve

calibrated_svc = CalibratedClassifierCV(model_SVM, method='sigmoid', cv='prefit')

def plot_roc(predict_fn, X, y, label=None):
    fprs, tprs, t = roc_curve(y, predict_fn(X)[:,-1])
    
    # Plot the ROC
    plt.plot(fprs, tprs, label="ROC "+label)
    plt.xlabel("FPR = 1 - specificity")
    plt.ylabel("TPR = sensitivity")
    plt.legend()

auc_list = []

for name, model, X in [('lr', model_logistic, X_train_scaled[features_logistic]), 
                    ('lda', model_LDA, X_train_scaled[features_LDA]), 
                    ('qda', model_QDA, X_train_scaled[features_QDA]),
                    ('SVC', calibrated_svc,X_train_scaled[features_SVM]),
                    ('KNN-all', model_KNN_all, X_train),
                    ('KNN-numeric', model_KNN_numeric, X_train[numeric_features])
                   ]:
    model.fit(X, y_train) # sistemare questa parte, fitta i modelli prima in modo che sia tutto ok
    plot_roc(model.predict_proba, X, y_train, name)
    y_prob_pred = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y_train, y_prob_pred)
    auc_list.append(auc)
auc_list